![wordcloud](wordcloud.png)

As a Data Scientist working for a mobile app company, you usually find yourself applying product analytics to better understand user behavior, uncover patterns, and reveal insights to identify the great and not-so-great features. Recently, the number of negative reviews has increased on Google Play, and as a consequence, the app's rating has been decreasing. The team has requested you to analyze the situation and make sense of the negative reviews.

It's up to you to apply K-means clustering from scikit-learn and NLP techniques through NLTK to sort text data from negative reviews in the Google Play Store into categories!

## The Data

A dataset has been shared with a sample of reviews and their respective scores (from 1 to 5) in the Google Play Store. A summary and preview are provided below.

# reviews.csv

| Column     | Description              |
|------------|--------------------------|
| `'content'` | Content (text) of each review. |
| `'score'` | Score assigned to the review by the user as an integer (from 1 to 5). |

In [50]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [51]:
# punkt -> สำหรับการแบ่งคำ (Tokenization)
# stopwords -> สำหรับการลบคำที่ไม่จำเป็น
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/repl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
# โหลดไฟล์ CSV 
reviews = pd.read_csv("reviews.csv")
reviews.head()

,content,score
0,I cannot open the app anymore,1
1,I have been begging for a refund from this app...,1
2,Very costly for the premium version (approx In...,1
3,"Used to keep me organized, but all the 2020 UP...",1
4,Dan Birthday Oct 28,1


In [53]:
# ขั้นตอนที่ 1: ประมวลผลรีวิวเชิงลบ (negative reviews) 

# เลือกรีวิวที่มีคะแนน 1 หรือ 2 (negative reviews) 
negative_reviews_tmp = reviews[(reviews["score"] == 1) | (reviews["score"] == 2)]["content"]

def preprocess_text(text):
    """Performs all the required steps in the text preprocessing"""

    # แบ่งคำ (Tokenizing) จากข้อความ
    tokens = word_tokenize(text)

    # ลบคำที่ไม่จำเป็น (stop words) และคำที่ไม่ใช่ตัวอักษร
    filtered_tokens = [
        token
        for token in tokens
        if token.isalpha() and token.lower() not in stopwords.words("english")
    ]

    return " ".join(filtered_tokens)

# ใช้ฟังก์ชันประมวลผลข้อความกับรีวิวเชิงลบ
negative_reviews_cleaned = negative_reviews_tmp.apply(preprocess_text)

# เก็บรีวิวที่ผ่านการประมวลผลใน DataFrame
preprocessed_reviews = pd.DataFrame({"review": negative_reviews_cleaned})
preprocessed_reviews.head()

,review
0,open app anymore
1,begging refund app month nobody replying
2,costly premium version approx Indian Rupees pe...
3,Used keep organized UPDATES made mess things c...
4,Dan Birthday Oct


In [54]:
# ขั้นตอนที่ 2: แปลงรีวิวที่ผ่านการประมวลผลเป็นเวกเตอร์ด้วยวิธี TF-IDF

# สร้างตัวแปลง (Vectorizer) สำหรับ TF-IDF
vectorizer = TfidfVectorizer()

# ใช้ฟังก์ชัน fit_transform เพื่อแปลงรีวิวที่ผ่านการประมวลผลเป็นเวกเตอร์ TF-IDF โดยจะสร้างเมทริกซ์ TF-IDF ที่ประกอบไปด้วยข้อมูลคำและความสำคัญของคำในแต่ละรีวิว
tfidf_matrix = vectorizer.fit_transform(preprocessed_reviews["review"])



In [56]:
# ขั้นตอนที่ 3: ใช้ K-means clustering เพื่อจัดกลุ่มข้อความใน tfidf_matrix

# ใช้ K-means clustering เพื่อจัดกลุ่มข้อมูล (โดยกำหนดจำนวนกลุ่มเป็น 5)
clust_kmeans = KMeans(n_clusters=5, random_state=500)
pred_labels = clust_kmeans.fit_predict(tfidf_matrix)

# เก็บ predicted labels เป็น list 
categories = pred_labels.tolist()
preprocessed_reviews["category"] = categories



In [57]:
# ขั้นตอนที่ 4: หาคำที่มีความถี่สูงสุด (Top term) สำหรับแต่ละกลุ่ม

# ดึง terms จาก vectorizer ที่ใช้แปลงข้อความเป็น TF-IDF
terms = vectorizer.get_feature_names_out()

# สร้าง list เพื่อเก็บข้อมูลคำที่มีความถี่สูงสุด (top term) สำหรับแต่ละกลุ่ม
topic_terms_list = []


# ทำการวนลูปผ่านแต่ละกลุ่มที่ได้จาก K-means
for cluster in range(clust_kmeans.n_clusters):
    # หาตำแหน่งของรีวิวที่อยู่ในกลุ่มปัจจุบัน
    cluster_indices = [i for i, label in enumerate(categories) if label == cluster]

    # คำนวณผลรวมของค่า tf-idf สำหรับแต่ละคำในกลุ่มนั้น ๆ
    cluster_tfidf_sum = tfidf_matrix[cluster_indices].sum(axis=0)
    cluster_term_freq = np.asarray(cluster_tfidf_sum).ravel()

    # หาคำที่มีน้ำหนักสูงสุด (top term) ในกลุ่มนี้
    top_term_index = cluster_term_freq.argsort()[::-1][0]

    # เพิ่มข้อมูลลงใน list ที่เก็บข้อมูลคำที่มีความถี่สูงสุด
    # - category: label / cluster assigned from K-means
    # - term: the identified top term
    # - frequency: term's weight for the category
    topic_terms_list.append(
        {
            "category": cluster,# label ที่ได้รับจาก K-means
            "term": terms[top_term_index],# คำที่มีน้ำหนักสูงสุดในกลุ่มนี้
            "frequency": cluster_term_freq[top_term_index],# ค่าน้ำหนักของคำในกลุ่มนี้
        }
    )


topic_terms = pd.DataFrame(topic_terms_list)
print(topic_terms)

   category      term   frequency
0         0       app  186.525216
1         1   version   63.738669
2         2      good   52.935519
3         3   premium   55.750426
4         4  calendar   70.971649
